In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [20]:
iis_logs_df = pd.read_csv("../data/u_ex181010.log"
            ,skiprows=[0,1,2,3]
            , sep=' ' 
            , usecols=[0,1, 2, 5, 6, 7, 8, 9, 10,11,12,14]
            , na_values='-'
            , names=[ 'date'
                    ,'time'
                    ,'server-ip'
                    ,'cs-uri-query'
                    ,'server-port'
                    ,'cs-username'
                    ,'client-ip'
                    ,'cs(User-Agent)'
                    ,'cs(Referer)'
                    ,'sc-status'
                    ,'sc-substatus'
                   ,'time-taken(ms)'])

In [21]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12338 entries, 0 to 12337
Data columns (total 12 columns):
date              12338 non-null object
time              12338 non-null object
server-ip         12337 non-null object
cs-uri-query      5512 non-null object
server-port       12335 non-null object
cs-username       7276 non-null object
client-ip         12335 non-null object
cs(User-Agent)    12139 non-null object
cs(Referer)       9207 non-null object
sc-status         12335 non-null object
sc-substatus      12335 non-null object
time-taken(ms)    12335 non-null object
dtypes: object(12)
memory usage: 1.1+ MB


In [22]:
iis_logs_df.describe()

,date,time,server-ip,cs-uri-query,server-port,cs-username,client-ip,cs(User-Agent),cs(Referer),sc-status,sc-substatus,time-taken(ms)
count,12338,12338,12337,5512,12335,7276,12335,12139,9207,12335,12335,12335
unique,5,6224,4,2718,3,85,223,91,314,7,2,1527
top,2018-10-10,02:56:42,192.168.2.210,ReturnUrl=%2f,443,s437875,137.92.20.64,Mozilla/5.0+(Windows+NT+10.0;+WOW64)+AppleWebK...,https://inplace.canberra.edu.au/Security/Accou...,200,0,46
freq,12334,27,12334,201,12311,1102,993,2289,1242,10581,12334,876
